# RPLib Problem 0001 - Baseline

Provides the baseline approach to rankability problem 0001. 

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [45]:
import copy
import os

import pandas as pd
import numpy as np

from scipy.stats import pearsonr

from tqdm import tqdm
#import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import joblib
import itertools
from pathlib import Path

from IPython.display import display, Markdown, Latex

**All packages are relative to the home directory of the user**

In [4]:
home = str(Path.home())

**Import the main rankability package**

In [7]:
import sys
sys.path.insert(0,"%s/rankability_toolbox_dev"%home)
import pyrankability

**Load the problem information**

In [8]:
problem = joblib.load("/disk/RPLib/problem_0001.joblib.z")

## Explore and setup the problem

In [9]:
problem.keys()

dict_keys(['description', 'target', 'data', 'other'])

In [10]:
print(problem["description"])


A practitioner wants to predict the degree to which a the rankings during season 
of the NCAA Men’s Basketball are likely to change as more games are played (i.e., sensitivity to more games). 
They want to start the analysis after a minimum of 50% of the games are played. 
They want to run Massey and Colley.

Sensitivity of new games will be measured as the intersection of between two 
rankings derived from before and after the new games are included.



In [11]:
problem['target']

,frac1,frac2,domain,range,direct_thres,spread_thres,weight_indirect,Method,Year,top15_intersection
0,0.5,0.6,all,madness,0.0,3.0,0.25,Massey,2002,0.733333
1,0.5,0.7,all,madness,0.0,3.0,0.25,Massey,2002,0.800000
2,0.5,0.8,all,madness,0.0,3.0,0.25,Massey,2002,0.733333
3,0.5,0.9,all,madness,0.0,3.0,0.25,Massey,2002,0.733333
4,0.5,1.0,all,madness,0.0,3.0,0.25,Massey,2002,0.600000
...,...,...,...,...,...,...,...,...,...,...
505,0.7,0.9,all,madness,3.0,3.0,0.00,Colley,2018,0.800000
506,0.7,1.0,all,madness,3.0,3.0,0.00,Colley,2018,0.733333
507,0.8,0.9,all,madness,3.0,3.0,0.00,Colley,2018,0.800000
508,0.8,1.0,all,madness,3.0,3.0,0.00,Colley,2018,0.733333


In [12]:
problem['data'].keys()

dict_keys(['2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018'])

In [13]:
problem['data']['2002'].keys()

dict_keys(['frac=0.5', 'frac=0.6', 'frac=0.7', 'frac=0.8', 'frac=0.9', 'frac=1.0'])

**Create easier to reference variables**

In [16]:
years = list(problem['data'].keys())
frac_keys = list(problem['data'][years[0]].keys())
remaining_games = problem['other']['remaining_games']
madness_teams = problem['other']['madness_teams']
best_df = problem['other']['best_df']
target_column = "top15_intersection"

## Define helper functions

**Function to compute a D matrix from games using hyperparameters**

In [17]:
def compute_D(game_df,team_range,direct_thres,spread_thres,weight_indirect):
    map_func = lambda linked: pyrankability.construct.support_map_vectorized_direct_indirect_weighted(linked,direct_thres=direct_thres,spread_thres=spread_thres,weight_indirect=weight_indirect)
    D = pyrankability.construct.V_count_vectorized(game_df,map_func).reindex(index=team_range,columns=team_range)
    return D

**Function to compute rankability metrics from D**

In [18]:
feature_columns = ["delta_lop","delta_hillside","nfrac_xstar_lop","nfrac_xstar_hillside","diameter_lop","diameter_hillside"]

def calc_tau(n,obj):
    nchoose2 = pyrankability.common.nCr(n,2)
    tau = (nchoose2 - obj)/nchoose2
    return tau

def compute_features(D,top_k=20):
    delta_lop,details_lop = pyrankability.rank.solve(D.fillna(0),method='lop',cont=True)

    x = pd.DataFrame(details_lop['x'],index=D.index,columns=D.columns)
    r = x.sum(axis=0)
    order = np.argsort(r)
    xstar = x.iloc[order,:].iloc[:,order]
    xstar.loc[:,:] = pyrankability.common.threshold_x(xstar.values)
    inxs = np.triu_indices(len(xstar),k=1)
    xstar_upper = xstar.values[inxs[0],inxs[1]]
    nfrac_upper_lop = sum((xstar_upper > 0) & (xstar_upper < 1))
    
    top_teams = xstar.columns[:top_k]
    
    k_two_distant,details_two_distant = pyrankability.search.solve_pair_max_tau(D.loc[top_teams,top_teams].fillna(0),method='lop',cont=False,verbose=False)
    d_lop = calc_tau(len(D),details_two_distant['obj'])
    
    delta_hillside,details_hillside = pyrankability.rank.solve(D,method='hillside',cont=True)
    
    x = pd.DataFrame(details_hillside['x'],index=D.index,columns=D.columns)
    r = x.sum(axis=0)
    order = np.argsort(r)
    xstar = x.iloc[order,:].iloc[:,order]
    xstar.loc[:,:] = pyrankability.common.threshold_x(xstar.values)
    inxs = np.triu_indices(len(xstar),k=1)
    xstar_upper = xstar.values[inxs[0],inxs[1]]
    nfrac_upper_hillside = sum((xstar_upper > 0) & (xstar_upper < 1))
    
    top_teams = xstar.columns[:top_k]
    
    k_two_distant,details_two_distant = pyrankability.search.solve_pair_max_tau(D.loc[top_teams,top_teams],method='hillside',verbose=False,cont=False)
    d_hillside = calc_tau(len(D),details_two_distant['obj'])
    
    features = pd.Series([delta_lop,delta_hillside,2*nfrac_upper_lop,2*nfrac_upper_hillside,d_lop,d_hillside],index=feature_columns)

    return features

In [33]:
def create_features(Ds):
    index_cols = list(Ds.index.names)
    X = pd.DataFrame(columns=index_cols + feature_columns)
    X.set_index(index_cols,inplace=True)
    for index,row in tqdm(Ds.iterrows()):
        year,frac_key,dt,st,iw,ran = index
        features = compute_features(Ds.loc[(year,frac_key,dt,st,iw,ran),"D"])
        features.name = index
        X = X.append(features)
    return X

In [28]:
def process(data,target,best_df):
    index_cols = ["Year","frac_key","direct_thres","spread_thres","weight_indirect","range"]
    Ds = pd.DataFrame(columns=["D"]+index_cols)
    Ds.set_index(index_cols,inplace=True)
    for frac_key,year in tqdm(itertools.product(frac_keys,years)):
        for index,row in best_df.iterrows():
            dom,ran,dt,st,iw = row.loc['domain'],row.loc['range'],row.loc['direct_thres'],row.loc['spread_thres'],row.loc['weight_indirect']
            # set the team_range
            team_range = None
            if ran == 'madness':
                team_range = madness_teams[year]
            elif ran == 'all':
                team_range = all_teams[year]
            else:
                raise Exception(f"range={ran} not supported")
            D = compute_D(data[year][frac_key],team_range,dt,st,iw)
            Ds = Ds.append(pd.Series([D],index=["D"],name=(year,frac_key,dt,st,iw,ran))) 
    return Ds

## Create D matrices

In [29]:
Ds = process(problem['data'],problem['target'],best_df)

102it [01:55,  2.01s/it]


In [30]:
Ds

D
Year frac_key direct_thres spread_thres weight_indirect range                                                     
2002 frac=0.5 0.0          3.0          0.25            madness  team2           Alabama  Alcorn_St  Arizona  B...
              3.0          3.0          0.00            madness  team2           Alabama  Alcorn_St  Arizona  B...
2003 frac=0.5 0.0          3.0          0.25            madness  team2        Alabama  Arizona  Arizona_St  Aub...
              3.0          3.0          0.00            madness  team2        Alabama  Arizona  Arizona_St  Aub...
2004 frac=0.5 0.0          3.0          0.25            madness  team2        Air_Force  Alabama  Alabama_St  A...
...                                                                                                            ...
2016 frac=1.0 3.0          3.0          0.00            madness  team2            Akron  Arizona  Ark_Little_Ro...
2017 frac=1.0 0.0          3.0          0.25            madness  team2          Arizona  Arkansas  Baylor  Buck...
              3.0          3.0          0.00            madness  team2          Arizona  Arkansas  Baylor  Buck...
2018 frac=1.0 0.0          3.0          0.25            madness  team2          Alabama  Arizona  Arkansas  Aub...
              3.0          3.0          0.00            madness  team2          Alabama  Arizona  Arkansas  Aub...

[204 rows x 1 columns]

In [31]:
Ds.loc[('2002','frac=0.5',0,3,0.25,'madness'),'D']

team2,Alabama,Alcorn_St,Arizona,Boston_College,Boston_Univ,California,Central_Conn,Charlotte,Cincinnati,Connecticut,...,UNC_Wilmington,USC,Utah,Valparaiso,W_Kentucky,Wake_Forest,Winthrop,Wisconsin,Wyoming,Xavier
team1,,,,,,,,,,,,,,,,,,,,,
Alabama,NaN,0.00,NaN,NaN,NaN,NaN,NaN,0.25,NaN,0.00,...,NaN,0.00,1.00,NaN,NaN,0.0,0.25,0.25,NaN,0.00
Alcorn_St,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.00,NaN,0.0,0.00,0.00,NaN,NaN
Arizona,NaN,NaN,0.00,NaN,NaN,0.0,NaN,0.50,0.0,0.25,...,NaN,0.25,0.00,1.25,NaN,NaN,0.00,NaN,NaN,0.25
Boston_College,NaN,NaN,NaN,0.00,1.00,0.0,0.25,NaN,NaN,0.25,...,0.00,0.00,NaN,NaN,NaN,NaN,0.25,NaN,NaN,0.00
Boston_Univ,NaN,NaN,NaN,0.00,NaN,0.0,0.00,0.00,NaN,0.00,...,NaN,NaN,NaN,0.00,NaN,NaN,0.00,0.00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wake_Forest,0.0,0.25,NaN,NaN,NaN,0.0,NaN,0.00,0.0,NaN,...,0.25,0.00,0.25,0.25,NaN,NaN,0.25,NaN,0.0,NaN
Winthrop,0.0,0.00,0.25,0.00,0.00,0.0,NaN,0.00,0.0,NaN,...,NaN,NaN,0.00,0.00,NaN,0.0,NaN,NaN,0.0,0.00
Wisconsin,0.0,0.25,NaN,NaN,0.25,NaN,NaN,0.00,0.0,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00


In [32]:
Ds.index.names

FrozenList(['Year', 'frac_key', 'direct_thres', 'spread_thres', 'weight_indirect', 'range'])

### To minimize computation time, limit to 0.5

In [36]:
X = create_features(Ds.xs('frac=0.5',level='frac_key',drop_level=False))

0it [00:00, ?it/s]

Using license file /home/jupyter-pander14/gurobi.lic
Academic license - for non-commercial use only - expires 2021-02-06


34it [02:12,  3.74s/it]


In [37]:
X

delta_lop  \
Year frac_key direct_thres spread_thres weight_indirect range                
2002 frac=0.5 0.0          3.0          0.25            madness      41.25   
              3.0          3.0          0.00            madness       9.00   
2003 frac=0.5 0.0          3.0          0.25            madness      24.00   
              3.0          3.0          0.00            madness       9.00   
2004 frac=0.5 0.0          3.0          0.25            madness      12.50   
              3.0          3.0          0.00            madness       2.00   
2005 frac=0.5 0.0          3.0          0.25            madness      12.50   
              3.0          3.0          0.00            madness       0.00   
2006 frac=0.5 0.0          3.0          0.25            madness      18.50   
              3.0          3.0          0.00            madness       2.00   
2007 frac=0.5 0.0          3.0          0.25            madness      28.50   
              3.0          3.0          0.00            madness       7.00   
2008 frac=0.5 0.0          3.0          0.25            madness      24.50   
              3.0          3.0          0.00            madness       4.00   
2009 frac=0.5 0.0          3.0          0.25            madness      20.50   
              3.0          3.0          0.00            madness       5.00   
2010 frac=0.5 0.0          3.0          0.25            madness      24.00   
              3.0          3.0          0.00            madness       4.00   
2011 frac=0.5 0.0          3.0          0.25            madness      29.50   
              3.0          3.0          0.00            madness       7.00   
2012 frac=0.5 0.0          3.0          0.25            madness      23.75   
              3.0          3.0          0.00            madness       8.00   
2013 frac=0.5 0.0          3.0          0.25            madness      19.75   
              3.0          3.0          0.00            madness       5.00   
2014 frac=0.5 0.0          3.0          0.25            madness      21.75   
              3.0          3.0          0.00            madness       5.00   
2015 frac=0.5 0.0          3.0          0.25            madness      24.50   
              3.0          3.0          0.00            madness       5.00   
2016 frac=0.5 0.0          3.0          0.25            madness      33.25   
              3.0          3.0          0.00            madness       7.00   
2017 frac=0.5 0.0          3.0          0.25            madness      13.50   
              3.0          3.0          0.00            madness       2.00   
2018 frac=0.5 0.0          3.0          0.25            madness      31.25   
              3.0          3.0          0.00            madness       4.00   

                                                                 delta_hillside  \
Year frac_key direct_thres spread_thres weight_indirect range                     
2002 frac=0.5 0.0          3.0          0.25            madness          8323.0   
              3.0          3.0          0.00            madness            48.0   
2003 frac=0.5 0.0          3.0          0.25            madness          5961.0   
              3.0          3.0          0.00            madness            33.0   
2004 frac=0.5 0.0          3.0          0.25            madness          5545.0   
              3.0          3.0          0.00            madness            10.0   
2005 frac=0.5 0.0          3.0          0.25            madness          5318.0   
              3.0          3.0          0.00            madness             6.0   
2006 frac=0.5 0.0          3.0          0.25            madness          7012.0   
              3.0          3.0          0.00            madness            36.0   
2007 frac=0.5 0.0          3.0          0.25            madness          8546.0   
              3.0          3.0          0.00            madness            38.0   
2008 frac=0.5 0.0          3.0          0.25            madness          6082.0   
           

## Refine the target dataset
We will try to predict the average over the parameters run for each method. In this problem, there is only a single set of parameters for each method, so this is technically unecessary. It is left to illustrate how this might be handed in other situations when averaging might be needed.

In [60]:
target = problem['target'].groupby(['frac1','frac2','Method','Year','direct_thres','spread_thres','weight_indirect'])[target_column].mean().to_frame()
target

top15_intersection
frac1 frac2 Method Year direct_thres spread_thres weight_indirect                    
0.5   0.6   Colley 2002 3.0          3.0          0.00                       0.800000
                   2003 3.0          3.0          0.00                       0.733333
                   2004 3.0          3.0          0.00                       0.800000
                   2005 3.0          3.0          0.00                       0.933333
                   2006 3.0          3.0          0.00                       0.800000
...                                                                               ...
0.9   1.0   Massey 2014 0.0          3.0          0.25                       0.800000
                   2015 0.0          3.0          0.25                       0.866667
                   2016 0.0          3.0          0.25                       0.800000
                   2017 0.0          3.0          0.25                       0.866667
                   2018 0.0          3.0          0.25                       0.800000

[510 rows x 1 columns]

In [61]:
X_for_join = X.copy().reset_index()
X_for_join['frac1']= X_for_join['frac_key'].str.replace("frac=","").astype(float)
X_for_join

,Year,frac_key,direct_thres,spread_thres,weight_indirect,range,delta_lop,delta_hillside,nfrac_xstar_lop,nfrac_xstar_hillside,diameter_lop,diameter_hillside,frac1
0,2002,frac=0.5,0.0,3.0,0.25,madness,41.25,8323.0,1682.0,296.0,0.967788,0.980288,0.5
1,2002,frac=0.5,3.0,3.0,0.00,madness,9.00,48.0,3338.0,2688.0,0.919712,0.917788,0.5
2,2003,frac=0.5,0.0,3.0,0.25,madness,24.00,5961.0,986.0,106.0,0.943452,0.982143,0.5
3,2003,frac=0.5,3.0,3.0,0.00,madness,9.00,33.0,3428.0,2652.0,0.916667,0.916667,0.5
4,2004,frac=0.5,0.0,3.0,0.25,madness,12.50,5545.0,1018.0,398.0,0.975962,0.997115,0.5
5,2004,frac=0.5,3.0,3.0,0.00,madness,2.00,10.0,3190.0,2844.0,0.925481,0.922115,0.5
6,2005,frac=0.5,0.0,3.0,0.25,madness,12.50,5318.0,1160.0,220.0,0.958173,0.992308,0.5
7,2005,frac=0.5,3.0,3.0,0.00,madness,0.00,6.0,3426.0,2826.0,0.920192,0.915385,0.5
8,2006,frac=0.5,0.0,3.0,0.25,madness,18.50,7012.0,528.0,196.0,0.979327,0.996154,0.5
9,2006,frac=0.5,3.0,3.0,0.00,madness,2.00,36.0,2926.0,2718.0,0.944231,0.925481,0.5


In [62]:
target

top15_intersection
frac1 frac2 Method Year direct_thres spread_thres weight_indirect                    
0.5   0.6   Colley 2002 3.0          3.0          0.00                       0.800000
                   2003 3.0          3.0          0.00                       0.733333
                   2004 3.0          3.0          0.00                       0.800000
                   2005 3.0          3.0          0.00                       0.933333
                   2006 3.0          3.0          0.00                       0.800000
...                                                                               ...
0.9   1.0   Massey 2014 0.0          3.0          0.25                       0.800000
                   2015 0.0          3.0          0.25                       0.866667
                   2016 0.0          3.0          0.25                       0.800000
                   2017 0.0          3.0          0.25                       0.866667
                   2018 0.0          3.0          0.25                       0.800000

[510 rows x 1 columns]

In [64]:
Xy = target.reset_index().set_index(['frac1','Year','direct_thres','spread_thres','weight_indirect']).join(X_for_join.set_index(['frac1','Year','direct_thres','spread_thres','weight_indirect'])).dropna()
Xy

frac2  Method  \
frac1 Year direct_thres spread_thres weight_indirect                  
0.5   2002 0.0          3.0          0.25               0.6  Massey   
                                     0.25               0.7  Massey   
                                     0.25               0.8  Massey   
                                     0.25               0.9  Massey   
                                     0.25               1.0  Massey   
...                                                     ...     ...   
      2018 3.0          3.0          0.00               0.6  Colley   
                                     0.00               0.7  Colley   
                                     0.00               0.8  Colley   
                                     0.00               0.9  Colley   
                                     0.00               1.0  Colley   

                                                      top15_intersection  \
frac1 Year direct_thres spread_thres weight_indirect                       
0.5   2002 0.0          3.0          0.25                       0.733333   
                                     0.25                       0.800000   
                                     0.25                       0.733333   
                                     0.25                       0.733333   
                                     0.25                       0.600000   
...                                                                  ...   
      2018 3.0          3.0          0.00                       0.800000   
                                     0.00                       0.733333   
                                     0.00                       0.733333   
                                     0.00                       0.733333   
                                     0.00                       0.600000   

                                                      frac_key    range  \
frac1 Year direct_thres spread_thres weight_indirect                      
0.5   2002 0.0          3.0          0.25             frac=0.5  madness   
                                     0.25             frac=0.5  madness   
                                     0.25             frac=0.5  madness   
                                     0.25             frac=0.5  madness   
                                     0.25             frac=0.5  madness   
...                                                        ...      ...   
      2018 3.0          3.0          0.00             frac=0.5  madness   
                                     0.00             frac=0.5  madness   
                                     0.00             frac=0.5  madness   
                                     0.00             frac=0.5  madness   
                                     0.00             frac=0.5  madness   

                                                      delta_lop  \
frac1 Year direct_thres spread_thres weight_indirect              
0.5   2002 0.0          3.0          0.25                 41.25   
                                     0.25                 41.25   
                                     0.25                 41.25   
                                     0.25                 41.25   
                                     0.25                 41.25   
...                                                         ...   
      2018 3.0          3.0          0.00                  4.00   
                                     0.00                  4.00   
                                     0.00                  4.00   
                                     0.00                  4.00   
                                     0.00                  4.00   

                                                      delta_hillside  \
frac1 Year direct_thres spread_thres weight_indirect                   
0.5   2002 0.0          3.0          0.25                     8323.0   
                                     0.25                     8323.0   
     

## Narrowing our goal to 0.5 and 0.6

In [65]:
data = Xy.reset_index().set_index(['frac1','frac2']).loc[0.5,0.6].reset_index()
data.head()

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


,frac1,frac2,Year,direct_thres,spread_thres,weight_indirect,Method,top15_intersection,frac_key,range,delta_lop,delta_hillside,nfrac_xstar_lop,nfrac_xstar_hillside,diameter_lop,diameter_hillside
0,0.5,0.6,2002,0.0,3.0,0.25,Massey,0.733333,frac=0.5,madness,41.25,8323.0,1682.0,296.0,0.967788,0.980288
1,0.5,0.6,2002,3.0,3.0,0.00,Colley,0.800000,frac=0.5,madness,9.00,48.0,3338.0,2688.0,0.919712,0.917788
2,0.5,0.6,2003,0.0,3.0,0.25,Massey,0.800000,frac=0.5,madness,24.00,5961.0,986.0,106.0,0.943452,0.982143
3,0.5,0.6,2003,3.0,3.0,0.00,Colley,0.733333,frac=0.5,madness,9.00,33.0,3428.0,2652.0,0.916667,0.916667
4,0.5,0.6,2004,0.0,3.0,0.25,Massey,0.866667,frac=0.5,madness,12.50,5545.0,1018.0,398.0,0.975962,0.997115


In [66]:
for_corr = data.set_index(['Method','direct_thres','spread_thres','weight_indirect'])
for ix in for_corr.index.unique():
    display(pd.Series(ix,index=for_corr.index.names))
    display(for_corr.loc[ix][[target_column]+feature_columns].corr())

Method             Massey
direct_thres            0
spread_thres            3
weight_indirect      0.25
dtype: object

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:4: PerformanceWarning: indexing past lexsort depth may impact performance.
  after removing the cwd from sys.path.


,top15_intersection,delta_lop,delta_hillside,nfrac_xstar_lop,nfrac_xstar_hillside,diameter_lop,diameter_hillside
top15_intersection,1.000000,-0.181122,-0.118407,0.067466,-0.087980,-0.079867,0.421510
delta_lop,-0.181122,1.000000,0.665603,0.354442,0.080729,-0.017145,-0.388255
delta_hillside,-0.118407,0.665603,1.000000,-0.118375,0.175304,0.495383,0.008037
nfrac_xstar_lop,0.067466,0.354442,-0.118375,1.000000,-0.025839,-0.578411,-0.350976
nfrac_xstar_hillside,-0.087980,0.080729,0.175304,-0.025839,1.000000,0.321597,-0.051514
diameter_lop,-0.079867,-0.017145,0.495383,-0.578411,0.321597,1.000000,0.315623
diameter_hillside,0.421510,-0.388255,0.008037,-0.350976,-0.051514,0.315623,1.000000


Method             Colley
direct_thres            3
spread_thres            3
weight_indirect         0
dtype: object

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:4: PerformanceWarning: indexing past lexsort depth may impact performance.
  after removing the cwd from sys.path.


,top15_intersection,delta_lop,delta_hillside,nfrac_xstar_lop,nfrac_xstar_hillside,diameter_lop,diameter_hillside
top15_intersection,1.000000,-0.045966,0.284547,-0.065937,0.058396,0.376589,0.102595
delta_lop,-0.045966,1.000000,0.721053,0.277524,-0.377616,0.097060,0.116633
delta_hillside,0.284547,0.721053,1.000000,-0.022799,-0.664217,0.495716,0.438286
nfrac_xstar_lop,-0.065937,0.277524,-0.022799,1.000000,-0.115375,-0.303857,-0.107203
nfrac_xstar_hillside,0.058396,-0.377616,-0.664217,-0.115375,1.000000,-0.293515,-0.452746
diameter_lop,0.376589,0.097060,0.495716,-0.303857,-0.293515,1.000000,0.689717
diameter_hillside,0.102595,0.116633,0.438286,-0.107203,-0.452746,0.689717,1.000000


In [72]:
for_corr = data.set_index(['Method','direct_thres','spread_thres','weight_indirect'])
for_display = pd.DataFrame(columns=feature_columns+list(for_corr.index.names))
for_display.set_index(list(for_corr.index.names),inplace=True)
for ix in for_corr.index.unique():
    dt = for_corr.loc[ix][[target_column]+feature_columns].corr().loc[target_column,feature_columns]
    dt.name = ix
    for_display = for_display.append(dt)

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  """


In [73]:
for_display

,,,,delta_lop,delta_hillside,nfrac_xstar_lop,nfrac_xstar_hillside,diameter_lop,diameter_hillside
Method,direct_thres,spread_thres,weight_indirect,,,,,,
Massey,0.0,3.0,0.25,-0.181122,-0.118407,0.067466,-0.087980,-0.079867,0.421510
Colley,3.0,3.0,0.00,-0.045966,0.284547,-0.065937,0.058396,0.376589,0.102595


In [76]:
print(for_display.T.to_latex())

\begin{tabular}{lrr}
\toprule
Method &    Massey &    Colley \\
direct\_thres &       0.0 &       3.0 \\
spread\_thres &       3.0 &       3.0 \\
weight\_indirect &      0.25 &      0.00 \\
\midrule
delta\_lop            & -0.181122 & -0.045966 \\
delta\_hillside       & -0.118407 &  0.284547 \\
nfrac\_xstar\_lop      &  0.067466 & -0.065937 \\
nfrac\_xstar\_hillside & -0.087980 &  0.058396 \\
diameter\_lop         & -0.079867 &  0.376589 \\
diameter\_hillside    &  0.421510 &  0.102595 \\
\bottomrule
\end{tabular}

